## This notebook finds the hmm states, each of which matches to some behavior exclusively

In [1]:
import os
import sys
sys.path.append('../libraries/') # Append the path within which the user-defined class is in
from data_loader import data_loader
from behavior_loader import bsoid_loader
import joblib
import pickle
import time

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from hmmlearn import hmm

## functions to load neural data

In [2]:
data_dir = r'../processed_data'
file_name = r'm1_fr0.1hz_30hz_0s_7200s_0.1s_bins_nooverlap.sav'
d_l = data_loader(data_dir, file_name)
data = d_l.load_data()

Spike times are binned into shape for hmm: (36000, 46)


## functions to load behavior

In [3]:
file_name = r'ag25290_day2_iter13'
# Load behaviors
behavior_loader = bsoid_loader(data_dir, file_name)
f_index, filtered_data, smoothed_predictions = behavior_loader.main()

mouse stayed in nest for 0.28108749470484795% of time
File #0 (a 6 body parts by 984391 frames) has 30 classes


In [4]:
# Correct prediction start
framerate = 60
delay = 6.8503
behavior_start = int(delay * framerate)  # start of behavior
smoothed_predictions_1h = smoothed_predictions[behavior_start:(behavior_start+3600*60)]

In [5]:
hmm_rate = 10
behaviors_10fps = smoothed_predictions_1h[0::int(framerate/hmm_rate)]

## load hmm and search for states that are exclusive to some behavior

In [18]:
model_dir = "../hmm_models/direct"

In [19]:
names_mod = ['insignificant', 'insignificant', 'insignificant', 'insignificant', 'rear',
         'insignificant', 'insignificant', 'investigate type 1', 'investigate type 2', 'contra-itch',
         'investigate type 3', 'insignificant', 'insignificant', 'insignificant', 'contra-body groom',
         'face groom type 1', 'dive/scrunch', 'head groom', 'ipsi-orient', 'insignificant',
         'face groom type 2', 'ipsi-body groom', 'ipsi-itch type 1', 'insignificant', 'insignificant', 
         'paw groom', 'locomotion', 'insignificant', 'contra-orient']

In [20]:
def ex_state_to_behavior(K):
    hmm_mrg = pickle.load(open(os.path.join(model_dir, f'1h_l5_possion_direct_{K}_latents.hmm'), 'rb'))
    hmm_states = hmm_mrg.predict(data)
    uiq_states = np.unique(hmm_states)
    for state in uiq_states:
        state_idcs = np.where(hmm_states == state)[0]
        behaviors = np.unique(behaviors_10fps[state_idcs])
        if len(behaviors) == 1:
            print(f'An exlusive state {state} of {len(state_idcs)} instances to behavior {names_mod[behaviors[0]]} found in 1h_l5_possion_merged_{K}_latents.hmm.')


In [21]:
ex_state_to_behavior(K=400)

In [22]:
ex_state_to_behavior(K=500)

An exlusive state 450 of 1 instances to behavior face groom type 1 found in 1h_l5_possion_merged_500_latents.hmm.
An exlusive state 461 of 4 instances to behavior contra-itch found in 1h_l5_possion_merged_500_latents.hmm.
